# `Setup`

In [1]:
import subprocess

try:
    subprocess.run(["docker", "inspect", "--format", "'{{.State.Running}}'", "redis"], check=True)
    print("Redis is already running.")
except subprocess.CalledProcessError:
    print("Starting Redis...")
    subprocess.run(["docker", "run", "--name", "redis", "-p", "6379:6379", "-d", "redis/redis-stack:latest"])
    print("Redis started.")


Starting Redis...
0800224747e1accd131b4141385bc275f2adf1cbeb7e9ae46336952f4597c452
Redis started.


Error: No such object: redis


# `Dict input, message output`

In [3]:
from typing import Optional

from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_community.chat_models import ChatOllama
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | ChatOllama(model="mistral")

In [5]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: RedisChatMessageHistory(session_id),
    input_messages_key="question",
    history_messages_key="history",
)

In [8]:
chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content=" In mathematics, particularly in the field of trigonometry, the cosine function is one of the three main functions along with sine and tangent. It is defined for non-negative angles θ as the ratio of the length of the adjacent side to the hypotenuse of a right triangle. In other words, if we have a right triangle with sides labeled opposite, adjacent, and hypotenuse (often shortened to OP, A, and H), then the cosine of θ (denoted as cos(θ)) is given by:\n\ncos(θ) = adjacent side / hypotenuse\n\nIn terms of angles in degrees or radians, the cosine function gives the ratio of the length of the adjacent side to the hypotenuse for a given angle. It can also be represented as a function of an angle using various mathematical notations such as cos(x) or C(x), where x is the angle in either degrees or radians.\n\nThe cosine function has some important properties: it is always between -1 and 1, it's even symmetric, meaning that cos(-x) = cos(x), and it oscillates between pos

In [9]:
chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content=' The inverse cosine function, denoted as arccos(x) or acos(x), is the inverse operation of the cosine function. It returns the angle (measured in degrees or radians) whose cosine value is a given input x. Since the cosine function outputs values between -1 and 1, its inverse, arccos(x), has a defined output only for inputs between -1 and 1. For other values of x, such as x < -1 or x > 1, there are two possible angles that could have given the input value as their cosine, making the inverse function ambiguous. In these cases, arccos(x) will return one of those valid angles based on a specific convention.\n\nThe arccosine function is defined as:\n\narccos(x) = θ, where cos(θ) = x and θ is in the first or fourth quadrants (i.e., θ lies between 0 and π/2 or π and 3π/2).\n\nThe main difference between the cosine function and its inverse is that the cosine function takes an angle as an input, while arccosine takes a cosine value as an input and returns the corresponding an

## `Messages input, dict output`

In [10]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOllama(model="mistral")})
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: RedisChatMessageHistory(session_id),
    output_messages_key="output_message",
)

chain_with_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content=' Simone de Beauvoir, a French philosopher and writer, is best known for her work on existentialist philosophy and feminist theory. In her philosophical works, she did not specifically focus on the topic of free will as extensively as some other philosophers have. However, she did touch upon this concept in relation to human freedom and existence.\n\nIn "The Second Sex," one of her most famous works, Beauvoir argues that women are not inherently different from men but are socially constructed as inferior. She also emphasizes the importance of individual freedom and choice, stating that individuals create their own meaning and purpose in life through their experiences and actions.\n\nIn this context, Beauvoir seems to lean toward the belief that humans possess free will, with the ability to make choices based on their unique circumstances and consciousness. However, she also acknowledges the role of social conditioning and societal expectations in sh

In [11]:
chain_with_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content=' Simone de Beauvoir and Jean-Paul Sartre, both French philosophers, shared many philosophical views, particularly in their emphasis on existentialism and the concept of human freedom. While they agreed on the importance of free will, there are some differences in how they approached the topic.\n\nSartre, in his seminal work "Being and Nothingness," argues for the absolute freedom of consciousness, known as "radical freedom." He posits that humans are born into a world without inherent meaning or purpose and must create their own existence through their choices and actions. This radical freedom is not determined by any external factors, including biology or social conditioning.\n\nBeauvoir\'s views on free will can be seen as influenced by Sartre\'s ideas but with a feminist perspective. In "The Second Sex," she acknowledges the role of individual choice and consciousness but also emphasizes the importance of understanding how social conditioning an

## `With itemgetter`

In [12]:
from operator import itemgetter

RunnableWithMessageHistory(
    ChatOllama(model="mistral"),
    lambda session_id: RedisChatMessageHistory(session_id),
)

RunnableWithMessageHistory(
    itemgetter("input_messages") | ChatOllama(model="mistral"),
    lambda session_id: RedisChatMessageHistory(session_id),
    input_messages_key="input_messages",
)

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  input_messages: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=RunnableLambda(itemgetter('input_messages'))
  | ChatOllama(model='mistral'), config_factories=[<function Runnable.with_listeners.<locals>.<lambda> at 0x117cf85e0>]), config={'run_name': 'RunnableWithMessageHistory'}), get_session_history=<function <lambda> at 0x117cf8700>, input_messages_key='input_messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

# `Tear down`

In [13]:
try:
    subprocess.run(["docker", "inspect", "--format", "'{{.State.Running}}'", "redis"], check=True)
    subprocess.run(["docker", "stop", "redis"])
    subprocess.run(["docker", "rm", "redis"])
    print("Redis has been stopped and removed.")

except subprocess.CalledProcessError:
    print("Redis is not currently running.")

'true'
redis
redis
Redis has been stopped and removed.
